In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from typing import List, Tuple
import os

In [ ]:
def create_folder_if_not_exists(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Folder {folder_path} created successfully.")
    else:
        print(f"Folder {folder_path} already exists.")


create_folder_if_not_exists("plots")
create_folder_if_not_exists("errors")

In [30]:
def get_A1(n, variable_type):
    A_ = np.zeros((n, n), dtype=variable_type)
    N = np.zeros(n, dtype=variable_type)
    for i in range(n):
        tmp = variable_type(i)
        N[i] = tmp
    for i in N:
        for j in N:
            index_i = int(i)
            index_j = int(j)
            if i - variable_type(1) < 0.001:
                A_[index_i][index_j] = variable_type(1)
            else:
                one = variable_type(1)
                A_[index_i][index_j] = one / ((i + one) + (j + one) - one)
    return A_


get_A1(5, np.float64)


array([[1.        , 1.        , 1.        , 1.        , 1.        ],
       [1.        , 1.        , 1.        , 1.        , 1.        ],
       [0.33333333, 0.25      , 0.2       , 0.16666667, 0.14285714],
       [0.25      , 0.2       , 0.16666667, 0.14285714, 0.125     ],
       [0.2       , 0.16666667, 0.14285714, 0.125     , 0.11111111]])

In [33]:
def get_A2(n, variable_type):
    A_ = np.zeros((n, n), dtype=variable_type)
    N = np.zeros(n, dtype=variable_type)
    one = variable_type(1)
    two = variable_type(2)
    for i in range(n):
        tmp = variable_type(i)
        N[i] = tmp
    for i in N:
        for j in N:
            index_i = int(i)
            index_j = int(j)
            if j >= i:
                A_[index_i][index_j] = (two * (i + one)) / (j + one)
            else:
                A_[index_i][index_j] = (two * (j + one)) / (i + one)
    return A_
get_A2(5, np.float64)

array([[2.        , 1.        , 0.66666667, 0.5       , 0.4       ],
       [1.        , 2.        , 1.33333333, 1.        , 0.8       ],
       [0.66666667, 1.33333333, 2.        , 1.5       , 1.2       ],
       [0.5       , 1.        , 1.5       , 2.        , 1.6       ],
       [0.4       , 0.8       , 1.2       , 1.6       , 2.        ]])

In [ ]:
def get_random_sign():
    return np.random.choice([-1, 1])


def get_x(n, variable_type):
    x_ = np.zeros(n, dtype=variable_type)
    for i in range(n):
        x_[i] = variable_type(variable_type(get_random_sign()))
    return x_

In [ ]:
import csv


def save_to_csv(filename, data):
    filename = "errors/" + filename + ".csv"
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(data)

In [ ]:
def get_b(A, x, n, variable_type):
    b = np.zeros(n, dtype=variable_type)
    for i in range(n):
        for j in range(n):
            b[i] += A[i][j] * x[j]
        print()
    return b

In [ ]:
#write a function which is given a matrix A and a vector b and returns the solution vector x
def gauss(A, b, variable_type):
    n = len(A)
    x = np.zeros(n, dtype=variable_type)
    for i in range(n):
        for j in range(i + 1, n):
            if A[i][i] == variable_type(0):
                print("Division by 0")
                return
            m = A[j][i] / A[i][i]
            for k in range(i, n):
                A[j][k] -= m * A[i][k]
            b[j] -= m * b[i]
    for i in range(n - 1, -1, -1):
        if A[i][i] == variable_type(0):
            print("Division by 0")
            return
        x[i] = b[i] / A[i][i]
        for j in range(i - 1, -1, -1):
            b[j] -= A[j][i] * x[i]
    return x

**Wykonanie*